In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [2]:
class LinearClassifier(nn.Module):
    def __init__(self, learning_rate, num_classes, input_dimensions):
        super(LinearClassifier, self).__init__()
        
        self.fully_connected_1 = nn.Linear(*input_dimensions, 128)
        self.fully_connected_2 = nn.Linear(*128, 256)
        self.fully_connected_3 = nn.Linear(*256, num_classes)
        
        self.optimizer = optim.Adam(self.parameters(), lr = learning_rate)
        self.loss = nn.CrossEntropyLoss()
        
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, data):
        layer1_output = F.sigmoid(self.fully_connected_1(data))
        layer2_output = F.sigmoid(self.fully_connected_2(layer1_output))
        layer3_output = self.fully_connected_3(layer2_output)
        
        return layer3_output
    
    def learn(self, data, labels):
        self.optimizer.zero_grad()
        data = T.tensor(data).to(self.device)
        labels = T.tensor(labels).to(self.device)
        
        predictions = self.forward(data)
        
        cost = self.loss(predictions, labels)
        
        cost.backward()
        self.optimizer.step()
        

In [3]:
lc = LinearClassifier(0.001, 4, 4)

TypeError: Value after * must be an iterable, not int